In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import timedelta,datetime
df=pd.read_csv('C:\\workplace\\electricity_demand\\dataset\\AEP_hourly.csv')
df.columns=['date_time','aep_mw']
df['date_time']=pd.to_datetime(df['date_time'])
df.sort_values('date_time',ascending=True,inplace=True)
df['date_time']=df['date_time'].astype(str)

In [3]:
def preparion_train (df,num_lags,lag_feature):

    df_lags=pd.DataFrame(index=df.index,columns=['lags_{}'.format(num_lags-1-i) for i in range(num_lags)])
    for i in range(num_lags):
        df_lags['lags_{}'.format(num_lags-1-i)]=df[lag_feature].shift(num_lags-1-i)
    
    df_targets=pd.DataFrame(index=df.index,columns=['step_{}'.format(i) for i in range(16,40)])
    for i in range(16,40):
        df_targets['step_{}'.format(i)]=df[lag_feature].shift(-i)
    
    df_processed=pd.concat([df[['date_time']],df_targets,df_lags],axis=1)
    #df_processed=df_processed[pd.to_datetime(df_processed['date_time']).dt.hour==trigerred_time]
    return df_processed.dropna()

# Train and Test split

In [4]:
from sklearn.preprocessing import PowerTransformer
scaler=PowerTransformer().fit(df.loc[df['date_time']<='2017-08-02 23:00:00',['aep_mw']])
df['scaled_aep_mw']=pd.Series(scaler.transform(df[['aep_mw']]).reshape(-1),index=df.index)
df.head()

,date_time,aep_mw,scaled_aep_mw
2183,2004-10-01 01:00:00,12379.0,-1.276695
2184,2004-10-01 02:00:00,11935.0,-1.492740
2185,2004-10-01 03:00:00,11692.0,-1.614218
2186,2004-10-01 04:00:00,11597.0,-1.662359
2187,2004-10-01 05:00:00,11681.0,-1.619773


In [5]:
df_processed=preparion_train (df,24,'scaled_aep_mw')
df_processed

,date_time,step_16,step_17,step_18,step_19,step_20,step_21,step_22,step_23,step_24,...,lags_9,lags_8,lags_7,lags_6,lags_5,lags_4,lags_3,lags_2,lags_1,lags_0
2206,2004-10-02 00:00:00,-0.727496,-0.709554,-0.769709,-0.815761,-0.620600,-0.521846,-0.539261,-1.013958,-1.376492,...,0.154540,0.154540,0.118224,0.005671,-0.120053,-0.049986,0.004110,-0.198959,-0.517181,-0.919724
2159,2004-10-02 01:00:00,-0.709554,-0.769709,-0.815761,-0.620600,-0.521846,-0.539261,-1.013958,-1.376492,-1.741196,...,0.154540,0.118224,0.005671,-0.120053,-0.049986,0.004110,-0.198959,-0.517181,-0.919724,-1.333872
2160,2004-10-02 02:00:00,-0.769709,-0.815761,-0.620600,-0.521846,-0.539261,-1.013958,-1.376492,-1.741196,-2.077776,...,0.118224,0.005671,-0.120053,-0.049986,0.004110,-0.198959,-0.517181,-0.919724,-1.333872,-1.624322
2161,2004-10-02 03:00:00,-0.815761,-0.620600,-0.521846,-0.539261,-1.013958,-1.376492,-1.741196,-2.077776,-2.191371,...,0.005671,-0.120053,-0.049986,0.004110,-0.198959,-0.517181,-0.919724,-1.333872,-1.624322,-1.788255
2162,2004-10-02 04:00:00,-0.620600,-0.521846,-0.539261,-1.013958,-1.376492,-1.741196,-2.077776,-2.191371,-2.281742,...,-0.120053,-0.049986,0.004110,-0.198959,-0.517181,-0.919724,-1.333872,-1.624322,-1.788255,-1.879762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116166,2018-08-01 05:00:00,0.422499,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,...,0.501312,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001
116167,2018-08-01 06:00:00,0.326766,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,...,0.458576,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482
116168,2018-08-01 07:00:00,-0.031118,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,...,0.347138,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927
116169,2018-08-01 08:00:00,-0.492645,-0.857230,-1.177993,-1.316519,-1.431649,-1.351272,-1.087313,-0.691662,-0.408344,...,0.064324,-0.367647,-0.745490,-1.030952,-1.216245,-1.359990,-1.317001,-1.118482,-0.756927,-0.486737


In [6]:
X_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('lags_')]]
X_train=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)#(X_train.shape[0],1,X_train.shape[1])
X_test=np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)#(X_test.shape[0],1,X_test.shape[1])

y_train=df_processed.loc[df_processed['date_time']<='2017-08-02 23:00:00',df_processed.columns[df_processed.columns.str.contains('step_')]]
y_test=df_processed.loc[(df_processed['date_time']>='2017-08-02 08:00:00') &\
    (pd.to_datetime(df_processed['date_time']).dt.hour==8),df_processed.columns[df_processed.columns.str.contains('step_')]]

# Hyperparameter tuning

In [7]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Input,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError

In [8]:
from sklearn.metrics import mean_absolute_percentage_error

In [9]:
import optuna

In [10]:
from sklearn.model_selection import train_test_split 
X_fit,X_val,y_fit,y_val=train_test_split(X_train,y_train,test_size=0.3,shuffle=False)

In [11]:
stoppage_rule=EarlyStopping(patience=10,monitor='val_loss')

In [12]:
def val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s):

    # Input layer
    model=Sequential()
    #model.add(Input(X_fit.shape[0],X_fit.shape[1],X_fit.shape[2]))
    # Hidden Layers
    for i in range(num_layers):
        if i==0:
            if num_layers>1:
                model.add(LSTM(units=num_nerons,return_sequences=True,input_shape=(X_fit.shape[1],X_fit.shape[2])))
            else:
                model.add(LSTM(units=num_nerons,return_sequences=False,input_shape=(X_fit.shape[1],X_fit.shape[2]))) 
        elif i<num_layers-1:
            model.add(LSTM(units=num_nerons,return_sequences=True))
        else :
            model.add(LSTM(units=num_nerons))
    # output layer
    model.add(Dense(units=24))
    # Training
    model.compile(optimizer=Adam(learning_rate=alpha),loss=MeanSquaredError())
    model.fit(X_fit,y_fit,validation_data=(X_val,y_val),epochs=iter,batch_size=b_s,callbacks=[stoppage_rule])
    # Evaluation of the model
    pred=model.predict(X_val)
    y_val2=scaler.inverse_transform(np.array(y_val).reshape(-1,1))
    pred2=scaler.inverse_transform(pred.reshape(-1,1))
    error=mean_absolute_percentage_error(y_val2,pred2)
    return error

In [13]:
def objective(trial):

    num_layers=trial.suggest_int('num_layers',1,2)
    num_nerons=trial.suggest_int('num_nerons',32,256,32)
    alpha=trial.suggest_float('alpha',0.001,0.1)
    iter=trial.suggest_int('iter',1000,1000)
    b_s=trial.suggest_int('b_s',32,256)
    loss_fun=val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s)
    return loss_fun 
study = optuna.create_study()
study.optimize(objective,n_trials=100)

[I 2024-02-18 00:38:02,056] A new study created in memory with name: no-name-b1b09d2e-0f8b-4144-9432-15b99e6ebeb0


Epoch 1/1000
443/443 [==============================] - 51s 105ms/step - loss: 0.6747 - val_loss: 0.3833
Epoch 2/1000
443/443 [==============================] - 40s 90ms/step - loss: 0.3390 - val_loss: 0.8392
Epoch 3/1000
443/443 [==============================] - 45s 102ms/step - loss: 0.3190 - val_loss: 0.3360
Epoch 4/1000
443/443 [==============================] - 50s 112ms/step - loss: 0.2992 - val_loss: 0.4103
Epoch 5/1000
443/443 [==============================] - 57s 128ms/step - loss: 0.3013 - val_loss: 0.3091
Epoch 6/1000
443/443 [==============================] - 53s 119ms/step - loss: 0.2922 - val_loss: 0.3072
Epoch 7/1000
443/443 [==============================] - 48s 109ms/step - loss: 0.2984 - val_loss: 0.3485
Epoch 8/1000
443/443 [==============================] - 48s 108ms/step - loss: 0.2953 - val_loss: 0.3289
Epoch 9/1000
443/443 [==============================] - 48s 109ms/step - loss: 0.3128 - val_loss: 0.3876
Epoch 10/1000
443/443 [==============================] -

[I 2024-02-18 00:51:27,141] Trial 0 finished with value: 0.08044329292284702 and parameters: {'num_layers': 1, 'num_nerons': 224, 'alpha': 0.05928558769698395, 'iter': 1000, 'b_s': 178}. Best is trial 0 with value: 0.08044329292284702.


Epoch 1/1000
1832/1832 [==============================] - 126s 64ms/step - loss: 0.2584 - val_loss: 0.2576
Epoch 2/1000
1832/1832 [==============================] - 115s 63ms/step - loss: 0.1956 - val_loss: 0.2566
Epoch 3/1000
1832/1832 [==============================] - 113s 62ms/step - loss: 0.1799 - val_loss: 0.2323
Epoch 4/1000
1832/1832 [==============================] - 112s 61ms/step - loss: 0.1706 - val_loss: 0.2316
Epoch 5/1000
1832/1832 [==============================] - 112s 61ms/step - loss: 0.1647 - val_loss: 0.2402
Epoch 6/1000
1832/1832 [==============================] - 112s 61ms/step - loss: 0.1584 - val_loss: 0.2136
Epoch 7/1000
1832/1832 [==============================] - 117s 64ms/step - loss: 0.1576 - val_loss: 0.2139
Epoch 8/1000
1832/1832 [==============================] - 115s 63ms/step - loss: 0.1512 - val_loss: 0.2180
Epoch 9/1000
1832/1832 [==============================] - 115s 63ms/step - loss: 0.1450 - val_loss: 0.2169
Epoch 10/1000
1832/1832 [============

[I 2024-02-18 01:21:59,972] Trial 1 finished with value: 0.06423079674325022 and parameters: {'num_layers': 2, 'num_nerons': 96, 'alpha': 0.01579481312416896, 'iter': 1000, 'b_s': 43}. Best is trial 1 with value: 0.06423079674325022.


...

Epoch 1/1000
431/431 [==============================] - 75s 154ms/step - loss: 0.2763 - val_loss: 0.3016
Epoch 2/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1967 - val_loss: 0.2519
Epoch 3/1000
431/431 [==============================] - 65s 150ms/step - loss: 0.1823 - val_loss: 0.2651
Epoch 4/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1697 - val_loss: 0.2570
Epoch 5/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1626 - val_loss: 0.2186
Epoch 6/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1577 - val_loss: 0.2368
Epoch 7/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1535 - val_loss: 0.2217
Epoch 8/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1486 - val_loss: 0.2179
Epoch 9/1000
431/431 [==============================] - 64s 149ms/step - loss: 0.1432 - val_loss: 0.2298
Epoch 10/1000
431/431 [==============================] 

[I 2024-02-18 08:12:27,617] Trial 16 finished with value: 0.06697957909742486 and parameters: {'num_layers': 2, 'num_nerons': 128, 'alpha': 0.022696929142328286, 'iter': 1000, 'b_s': 183}. Best is trial 11 with value: 0.06124636174458179.


Epoch 1/1000
536/536 [==============================] - 39s 57ms/step - loss: 0.2699 - val_loss: 0.2770
Epoch 2/1000
536/536 [==============================] - 28s 53ms/step - loss: 0.2047 - val_loss: 0.2820
Epoch 3/1000
536/536 [==============================] - 28s 52ms/step - loss: 0.1917 - val_loss: 0.2525
Epoch 4/1000
536/536 [==============================] - 28s 53ms/step - loss: 0.1820 - val_loss: 0.2329
Epoch 5/1000
536/536 [==============================] - 28s 53ms/step - loss: 0.1779 - val_loss: 0.2325
Epoch 6/1000
536/536 [==============================] - 28s 53ms/step - loss: 0.1720 - val_loss: 0.2244
Epoch 7/1000
536/536 [==============================] - 32s 60ms/step - loss: 0.1696 - val_loss: 0.2254
Epoch 8/1000
536/536 [==============================] - 29s 54ms/step - loss: 0.1669 - val_loss: 0.2492
Epoch 9/1000
536/536 [==============================] - 28s 53ms/step - loss: 0.1642 - val_loss: 0.2369
Epoch 10/1000
536/536 [==============================] - 28s 53m

[W 2024-02-18 08:19:07,474] Trial 17 failed with parameters: {'num_layers': 2, 'num_nerons': 32, 'alpha': 0.038704368138115966, 'iter': 1000, 'b_s': 147} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\shaha\anaconda3\envs\aep_hourly\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\shaha\AppData\Local\Temp\ipykernel_5952\2204108411.py", line 8, in objective
    loss_fun=val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s)
  File "C:\Users\shaha\AppData\Local\Temp\ipykernel_5952\3712466499.py", line 21, in val_fun
    model.fit(X_fit,y_fit,validation_data=(X_val,y_val),epochs=iter,batch_size=b_s,callbacks=[stoppage_rule])
  File "c:\Users\shaha\anaconda3\envs\aep_hourly\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\shaha\anaconda3\envs\aep_hourly\lib\site-packages\k

KeyboardInterrupt: 

In [16]:
# {'num_layers': 2, 'num_nerons': 32, 'alpha': 0.0030574682781916596, 'iter': 25, 'b_s': 254}.
# Input layer
model=Sequential()

#model.add(Input(X_fit.shape[1],X_fit.shape[2]))
model.add(LSTM(32,input_shape=(24,1),return_sequences=True))
model.add(LSTM(32))
model.add(Dense(24))
model.summary()
model.compile(optimizer=Adam(learning_rate=0.0030574682781916596),loss=MeanSquaredError())
model.fit(X_fit,y_fit,epochs=25,batch_size=254)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_35 (LSTM)              (None, 24, 32)            4352      
                                                                 
 lstm_36 (LSTM)              (None, 32)                8320      
                                                                 
 dense_20 (Dense)            (None, 24)                792       
                                                                 
Total params: 13,464
Trainable params: 13,464
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
311/311 [==============================] - 19s 45ms/step - loss: 0.3873
Epoch 2/25
311/311 [==============================] - 24s 78ms/step - loss: 0.2602
Epoch 3/25
311/311 [==============================] - 20s 63ms/step - loss: 0.2269
Epoch 4/25
311/311 [==============================] - 20s 64ms/s

In [17]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_35 (LSTM)              (None, 24, 32)            4352      
                                                                 
 lstm_36 (LSTM)              (None, 32)                8320      
                                                                 
 dense_20 (Dense)            (None, 24)                792       
                                                                 
Total params: 13,464
Trainable params: 13,464
Non-trainable params: 0
_________________________________________________________________


In [18]:
pred=model.predict(X_test)

12/12 [==============================] - 2s 22ms/step


In [19]:
y_test2=scaler.inverse_transform(np.array(y_test).reshape(-1,1))
pred2=scaler.inverse_transform(pred.reshape(-1,1))
error=mean_absolute_percentage_error(y_test2,pred2)
error

0.06805593342685959

In [20]:
pred.reshape(-1)

array([-0.16380438, -0.6046224 , -0.9742418 , ...,  0.63043994,
        0.41948518,  0.1537948 ], dtype=float32)

In [21]:
pred2

array([[14924.479],
       [13862.149],
       [13026.863],
       ...,
       [17035.406],
       [16448.773],
       [15737.06 ]], dtype=float32)

In [22]:
pred2=pred.reshape(-1,1)
pred2=scaler.inverse_transform(pred2)
pred2=pd.Series(pred2.reshape(-1))

actual=df.loc[df['date_time']>='2017-08-03 00:00:00','aep_mw']
actual.index=range(len(actual))
actual.drop(8760,inplace=True)
pred_df=pd.concat([actual,pred2],axis=1)
pred_df.columns=['aep_mw','forecast']
pred_df.index=df.loc[df['date_time']>='2017-08-03 00:00:00']['date_time'].iloc[:8760]

In [23]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(actual,pred2)

0.06869809188830196

In [24]:
import plotly.graph_objects as go
figure=go.Figure()
figure.add_trace(go.Scatter(x=df['date_time'].tail(len(pred2)),y=pred_df['aep_mw'],name='actual_electricity_load'))
figure.add_trace(go.Scatter(x=df['date_time'].tail(len(pred2)),y=pred_df['forecast'],name='mlp_forecast'))